## 1️⃣ Contexto actual

En la clase 16 creamos y ejecutamos el contenedor de la **API**.

Estructura actual del proyecto:
```
app/
├─ api/
│  ├─ main.py
│  ├─ requirements.txt
│  └─ Dockerfile
└─ ui/
   └─ app.py
```

Ahora vamos a:
1. Construir la imagen de la UI (Streamlit).  
2. Ejecutarla junto a la API.  
3. Ver que la UI **no puede conectarse** cuando llama a `http://127.0.0.1:8000/api/v1/predict`.

## Cambios:
### Scrip `app/UI/main.py`
-  Vamos a actualizar la siguiente línea para permitir valores decimales en el campo `trip_distance` 
```python
trip_distance = st.sidebar.number_input("Trip Distance", value=10.0, min_value=0.1, max_value=100.0)
```

## 2️⃣ Dockerfile de la UI

Coloca este archivo en `app/ui/Dockerfile`:

```Dockerfile
FROM python:3.13-slim

WORKDIR /code

RUN apt-get update && apt-get install -y procps && \
    pip install -U pip && \
    rm -rf /var/lib/apt/lists/*

COPY ./requirements.txt /code/requirements.txt

RUN pip install -r ./requirements.txt

COPY ./main.py /code/

EXPOSE 8501

CMD ["streamlit", "run", "main.py", "--server.port=8501", "--server.address=0.0.0.0"]
```

## 3️⃣ Construcción de la imagen

Ejecuta desde la raíz del proyecto:

```bash
docker build -t nyc-taxi-ui-image app/ui/
```

Esto genera la imagen `nyc-taxi-ui-image` que ejecutará Streamlit en el puerto 8501.

## 4️⃣ Ejecutar API y UI (separadas)

1️⃣ Inicia la API:
```bash
docker run -p 8000:8000 --env-file .env --name nyc-taxi-api-container nyc-taxi-api-image
```

2️⃣ Inicia la UI:
```bash
docker run -p 8501:8501 --name nyc-taxi-ui-container nyc-taxi-ui-image
```

Abre [http://localhost:8501](http://localhost:8501)  
y presiona el botón **Predict**.


## 5️⃣ Qué ocurre al hacer click en Predict

Streamlit intenta hacer `POST` a:

```
http://127.0.0.1:8000/api/v1/predict
```

🧩 **Importante:**  
`127.0.0.1` es la dirección IP de **localhost**, es decir, **la propia máquina donde se ejecuta el código**.

---

### 🚫 Pero dentro de Docker…

Desde dentro del contenedor `nyc-taxi-ui-container`, `127.0.0.1` (o `localhost`) **no apunta al host ni a la API**, sino **al mismo contenedor de la UI**.

Por eso, al presionar el botón **Predict**, Streamlit intenta conectarse consigo mismo, y no con el contenedor que corre la API.

El resultado es el clásico error de conexión:

> ❌ `ConnectionError: Failed to establish a new connection: [Errno 111] Connection refused`

---

### 🧠 Analogía rápida

> Cada contenedor es como una **casa diferente**.  
> Decir “localhost” es como decir “mi casa”:  
> cada contenedor se está hablando **a sí mismo**, no al vecino.

Para que la UI y la API puedan comunicarse, necesitamos ponerlas **en el mismo vecindario de red**.

## 6️⃣ Cómo solucionar el problema de `localhost`

Ya vimos que la UI no puede hablar con la API porque, dentro de Docker, **cada contenedor tiene su propia red interna**.

Para solucionarlo debemos hacer que **ambos contenedores compartan la misma red**. Aquí es donde entra en juego una herramienta fundamental de Docker 🧠👇

# 🌐 Network en Docker

Docker Network es la funcionalidad que permite a los contenedores comunicarse entre sí y con el mundo exterior de forma controlada y segura.

Docker crea **redes virtuales** que conectan los contenedores, proporcionando aislamiento, seguridad y comunicación eficiente.

---

### 🔧 Tipos de redes en Docker

| Tipo | Descripción |
|------|--------------|
| **bridge** | Red por defecto. Permite que los contenedores conectados al mismo puente se comuniquen entre sí, pero los aísla de otros puentes. |
| **host** | Elimina el aislamiento entre los contenedores y el host, haciendo que compartan la red del sistema. Ideal para desarrollo local. |
| **overlay** | Permite la comunicación entre contenedores en distintos hosts. Usado en entornos distribuidos (Swarm, Kubernetes). |
| **macvlan** | Asigna una dirección MAC a cada contenedor, haciéndolo parecer un dispositivo físico en la red. |

---

> 🧠 **Idea principal:**  
> Docker Network actúa como un “vecindario virtual”.  
> Los contenedores conectados a la misma red pueden comunicarse usando su **nombre** en lugar de direcciones IP.
>
> En esta clase usaremos la red **del host** para simplificar el desarrollo y mantener la llamada a la API con `localhost`.

# ⚙️ Solución: usar `--network host`

El modo **host** permite que los contenedores compartan directamente la red del sistema anfitrión.  

De esta manera, `localhost` (o `127.0.0.1`) **funciona igual que fuera de Docker**.

---

### 🪟 macOS / Windows / Linux (Docker Desktop 4.34+)

> ⚙️ Abre **Docker Desktop → Settings → Resources → Network → Enable host networking**

Una vez activada esta opción, ejecuta:

```bash
# API
docker run --network host -p 8000:8000 --env-file .env --name nyc-taxi-api-container nyc-taxi-api-image

# UI
docker run --network host -p 8501:8501 --name nyc-taxi-ui-container nyc-taxi-ui-image
```

Ambos comparten la red del host y la UI puede seguir usando:

```python
url = "http://127.0.0.1:8000/api/v1/predict"
```

# 🧩 Introducción a Docker Compose

## ¿Qué es Docker Compose?

**Docker Compose** es una herramienta que permite definir y ejecutar aplicaciones **multi-contenedor** de forma sencilla.  

Utiliza un archivo `compose.yaml` o `docker-compose.yml` donde se describen todos los servicios, sus puertos, redes, volúmenes y variables de entorno.  

👉 En lugar de ejecutar varios `docker run` manualmente, puedes levantar **toda tu aplicación** con un solo comando.

---

## 💡🚀 ¿Por qué usar Docker Compose?

- **Facilidad de uso:** Levantas y detienes múltiples contenedores con un solo comando.  
- **Configuración centralizada:** Todo está documentado en un único archivo (`compose.yaml`).  
- **Escalabilidad:** Puedes aumentar réplicas de servicios fácilmente.  
- **Reproducibilidad:** El entorno se comporta igual en cualquier computadora.

---

## 🧩 Estructura básica de un archivo `docker-compose.yaml`

El archivo define:
- **services:** los contenedores que forman tu aplicación.  
- **ports:** los puertos que se exponen al host.  
- **build:** las rutas de los Dockerfiles.  
- **env_file:** archivos `.env` con variables de entorno.

Ejemplo genérico 👇

```yaml
services:
  frontend:
    build: 
      context: ./frontend
    ports:
      - "3000:3000"
    depends_on:
      - backend

  backend:
    build: 
      context: ./backend
    ports:
      - "8000:8000"
    env_file:
      - ./backend/.env
```

---

## 🚕 Ejemplo de nuestro proyecto *NYC Taxi Time Prediction*

Nuestra aplicación tiene **dos servicios**:
- Una **API** (FastAPI) que sirve el modelo de predicción.  
- Una **UI** (Streamlit) que envía solicitudes a la API.  

Además, la API necesita variables de entorno definidas en un archivo `.env` que vive en la raíz del proyecto.

---

### 🧱 Archivo `docker-compose.yaml`

Guarda este archivo en la raíz del proyecto:

```yaml
name: nyc-taxi-app  # Nombre del proyecto de la aplicación

services:
  # 🧩 Servicio de Frontend (UI - Streamlit)
  ui:
    image: nyc-taxi-ui-image:latest                # Nombre y etiqueta de la imagen
    container_name: nyc-taxi-ui-container          # Nombre asignado al contenedor
    build:
      context: ./app/ui                            # Carpeta donde está el código del frontend
      dockerfile: Dockerfile                       # Nombre del Dockerfile dentro del contexto
    ports:
      - "8501:8501"                                # Puerto UI → http://localhost:8501
    network_mode: host                             # Usa la red del host (localhost)
    command: streamlit run main.py --server.port=8501 --server.address=0.0.0.0

  # 🧠 Servicio de Backend (API - FastAPI)
  api:
    image: nyc-taxi-api-image:latest               # Nombre y etiqueta de la imagen
    container_name: nyc-taxi-api-container         # Nombre asignado al contenedor
    build:
      context: ./app/api                           # Carpeta donde está el código de la API
      dockerfile: Dockerfile                       # Nombre del Dockerfile dentro del contexto
    ports:
      - "8000:8000"                                # Puerto API → http://localhost:8000
    env_file:
      - ./.env                                     # Carga variables desde el archivo .env en la raíz
    network_mode: host                             # Usa la red del host (localhost)
    command: uvicorn api:app --host 0.0.0.0 --port 8000
```
---



## ⚙️ Comandos útiles de Docker Compose

### 🧱 1. Construir imágenes
```bash
docker compose build
```

### 🔁 2. Reconstruir sin usar caché
```bash
docker compose build --no-cache
```

### 🚀 3. Construir y ejecutar simultáneamente
```bash
docker compose up --build
```

### ▶️ 4. Ejecutar servicios (modo interactivo)
```bash
docker compose up
```

### 🧩 5. Ejecutar en segundo plano (modo “detached”)
```bash
docker compose up -d
```

### 🛑 6. Detener y eliminar servicios
```bash
docker compose down
```

### 🪵 7. Ver logs de todos los servicios
```bash
docker compose logs
```

### 🔍 8. Ver el estado de los contenedores
```bash
docker compose ps
```

---

## ✅ En resumen

| Acción | Comando | Descripción |
|--------|----------|-------------|
| **Construir imágenes** | `docker compose build` | Crea las imágenes necesarias |
| **Levantar entorno** | `docker compose up` | Inicia todos los servicios |
| **Ejecutar en background** | `docker compose up -d` | Ejecuta los servicios en segundo plano |
| **Detener entorno** | `docker compose down` | Detiene y elimina los contenedores |
| **Ver logs** | `docker compose logs` | Muestra la salida combinada de los contenedores |

---


| PULocationID | DOLocationID | trip_distance | duration          |
|--------------|--------------|---------------|--------------------|
| 236          | 239          | 1.98          | 11.5               |
| 65           | 170          | 6.54          | 20.87              |
| 74           | 262          | 3.08          | 19.03              |
| 74           | 116          | 2.40          | 11.87              |
| 74           | 243          | 5.14          | 10.98              |
| 33           | 209          | 2.00          | 16.70              |
| 74           | 238          | 3.20          | 16.22              |
| 166          | 239          | 2.01          | 11.45              |
| 226          | 226          | 0.31          | 1.27               |
| 7            | 129          | 2.32          | 13.38              |
| 42           | 75           | 2.69          | 16.65              |


# 🐳 Docker Hub y cómo subir imágenes a tu repositorio privado

## 🌍 ¿Qué es Docker Hub?

**Docker Hub** es un servicio en la nube que funciona como un **registro de imágenes**: un lugar donde puedes **almacenar, compartir y descargar** imágenes de Docker.  

Es el registro público más utilizado y ofrece:
- Repositorios **públicos** → gratuitos y visibles para todos.  
- Repositorios **privados** → acceso restringido (gratuitos con límite o por suscripción).

---

### 💡 ¿Qué puedes hacer con Docker Hub?

- 🔍 **Buscar** imágenes públicas creadas por la comunidad.  
- ☁️ **Subir** tus propias imágenes (públicas o privadas).  
- ⚙️ **Automatizar** despliegues y CI/CD integrándolo con GitHub o pipelines.

---

## 🚀 Pasos para subir imágenes a tu repositorio en Docker Hub

### 🧱 1. Crear una cuenta
1. Ve a [hub.docker.com](https://hub.docker.com).  
2. Regístrate o inicia sesión.  
3. Crea un nuevo repositorio (público o privado) desde la interfaz web.  
   - Clic en **Create Repository**  
   - Asigna un nombre (por ejemplo, `nyc-taxi-api`)  
   - Marca la opción **Private** si no quieres que sea visible públicamente.

---

### 🔑 2. Iniciar sesión desde la terminal

Para subir una imagen necesitas autenticarte:

```bash
docker login
```

👉 Ingresa tu **usuario** y **contraseña** (o token de acceso).  
Si la autenticación es correcta, podrás hacer *push* o *pull* desde tu cuenta.

---

### 🏷️ 3. Etiquetar tu imagen (`docker tag`)

Antes de subirla, debes “etiquetar” tu imagen con el formato:
```
<usuario-dockerhub>/<nombre-repo>:<tag>
```

Por ejemplo:
```bash
docker tag nyc-taxi-api-image:latest zapatacc/nyc-taxi-api-image:v1
```

📌 **Notas:**
- Si no especificas un `:tag`, Docker usará `:latest` por defecto.  
- El nombre del usuario (namespace) debe coincidir con tu cuenta de Docker Hub.  
- Puedes listar tus imágenes locales con:
  ```bash
  docker images
  ```

---

### ☁️ 4. Subir la imagen a Docker Hub (`docker push`)

Ahora envía la imagen etiquetada a tu repositorio remoto:

```bash
docker push zapatacc/nyc-taxi-api-image:v1
```

Docker subirá todas las capas de la imagen a tu cuenta en Docker Hub.  

El proceso puede tardar dependiendo del tamaño de la imagen y tu conexión.

---

### 🔎 5. Verificar la subida

Una vez finalizado el `push`:
- Ve a [hub.docker.com](https://hub.docker.com)
- Abre tu repositorio → deberías ver tu imagen con la etiqueta `v1`.
- Desde ahí puedes administrar permisos, visibilidad y versiones.

---

## 🔒 Repositorios privados

Puedes crear repositorios **privados** para imágenes que no deben ser públicas.

Pasos rápidos:
1. En Docker Hub → **Create Repository**  
2. Asigna un nombre (ejemplo: `nyc-taxi-ui`)  
3. Selecciona la opción **Private**  
4. Haz `docker tag` y `docker push` como en los pasos anteriores.

💡 En cuentas gratuitas, Docker Hub permite un número limitado de repositorios privados.

---

## 📦 Descargar imágenes desde Docker Hub

### 🔓 Repositorio público
Cualquiera puede descargarlo:
```bash
docker pull zapatacc/nyc-taxi-api:v1
```

### 🔒 Repositorio privado
El usuario debe iniciar sesión con sus credenciales antes:
```bash
docker login
docker pull zapatacc/nyc-taxi-api:v1
```

Si no está autenticado o no tiene permisos, recibirá un error de acceso denegado.

---

## 🧠 Resumen general

| Paso | Comando | Descripción |
|------|----------|-------------|
| 1️⃣ Login | `docker login` | Autenticarse en Docker Hub |
| 2️⃣ Etiquetar | `docker tag <imagen>:<tag> <usuario>/<repo>:<tag>` | Asignar formato correcto a la imagen |
| 3️⃣ Subir | `docker push <usuario>/<repo>:<tag>` | Enviar la imagen al repositorio remoto |
| 4️⃣ Descargar | `docker pull <usuario>/<repo>:<tag>` | Obtener la imagen desde Docker Hub |
| 5️⃣ Verificar | (web) [hub.docker.com](https://hub.docker.com) | Confirmar que la imagen aparece correctamente |

---

> 💬 **Tip para clase:**  
> “Piensen en Docker Hub como el *GitHub de las imágenes Docker*:  
> puedes hacer *push* (subir), *pull* (descargar) y compartir tus versiones.”  

# 🤗 Desplegar tu aplicación con Docker Compose en Hugging Face Spaces

## 💡 ¿Qué es Hugging Face?

**Hugging Face** es una plataforma de inteligencia artificial que permite **compartir modelos, datasets y aplicaciones interactivas**.  

Es ampliamente usada por la comunidad de Machine Learning para publicar y probar proyectos sin necesidad de infraestructura propia.

Entre sus productos más conocidos están:
- 🧠 **Transformers Hub** → repositorio de modelos de IA (NLP, CV, Audio, etc.)
- 📊 **Datasets Hub** → colección de conjuntos de datos públicos.
- 💻 **Spaces** → servicio gratuito para desplegar **aplicaciones web y demos** (Streamlit, Gradio, Docker, etc.)

---

## 🚀 ¿Qué es Hugging Face Spaces?

**Spaces** es el servicio de Hugging Face para **alojar y compartir apps interactivas**.  

Permite desplegar:
- Aplicaciones en **Streamlit**, **Gradio** o **Static HTML** directamente desde un repositorio Git.  
- O, más avanzado, **contenedores Docker** (incluso usando `docker compose`).
---

## 🧱 ¿Cómo funciona el despliegue con Docker?

Hugging Face Spaces ofrece un tipo de “Space” con **Runtime: Docker**.  

Esto significa que puedes subir tu propio `Dockerfile` o `docker-compose.yml` y el espacio se encargará de:

1. Construir las imágenes automáticamente (o descargarlas desde Docker Hub).  
2. Ejecutar los servicios definidos.  
3. Publicar la aplicación final con una URL pública.

---

## 🆓 Plan gratuito (Docker Spaces)

- ✅ 1 espacio **Docker gratuito por cuenta**.  
- 🔒 Recursos limitados: CPU básica y 16 GB de almacenamiento temporal.  
- 🕒 El espacio entra en modo “sleep” tras cierto tiempo inactivo.  
- 🚫 No permite persistencia de datos (se reinicia al reiniciar el contenedor).  
- 🧩 Ideal para proyectos de clase, demostraciones y portafolios.

---

# 🚀 Desplegar el proyecto en Hugging Face Spaces

En esta sección desplegaremos nuestra aplicación *NYC Taxi Time Prediction* en Hugging Face **creando dos Spaces**:

- `nyc-time-api-hf` → para la API (FastAPI)
- `nyc-time-ui-hf` → para la UI (Streamlit)

Primero desplegaremos la **API**, verificaremos que funcione, y luego haremos el **deploy de la UI** apuntando a la URL pública de la API.

## 1️⃣ Crear cuenta y obtener token de acceso

1. Ve a 👉 [https://huggingface.co/join](https://huggingface.co/join) y crea tu cuenta.  
2. Genera un **Access Token** con permisos de escritura:
   - En tu perfil: **Settings → Access Tokens → New token**
   - Marca los scopes `read` y `write`.
   - Copia el token (solo se muestra una vez) y guardalo en un bloc de notas temporal.

> 🔑 Usaremos este token para autenticarnos vía **HTTPS** cuando hagamos `git push`.la primera vez

## 2️⃣ Crear el Space de la API

1. Ve a 👉 [https://huggingface.co/spaces](https://huggingface.co/spaces)  
2. Clic en **Create new Space**
3. Nombre: `nyc-time-api-hf`
4. Runtime: **Docker**
5. Visibilidad: **Public** (o *Private* si prefieres)

El repositorio se creará en:

```
https://huggingface.co/spaces/<tu_usuario>/nyc-time-api-hf
```

Y la app pública quedará en:

```
https://<tu_usuario>-nyc-time-api-hf.hf.space
```


## 3️⃣ Clonar el Space de la API en local

Ejecuta en tu terminal:

```bash
git clone https://huggingface.co/spaces/<tu_usuario>/nyc-time-api-hf
cd nyc-time-api-hf
```

> 💡 La primera vez que hagas `git push`, Git te pedirá:
> - **Username:** tu usuario de Hugging Face  
> - **Password:** tu *Access Token* (no tu contraseña)

## 4️⃣ Copiar archivos del proyecto al Space de la API

Dentro del repositorio clonado (`nyc-time-api-hf/`), copia los siguientes tres archivos desde tu proyecto local:

- `Dockerfile`
- `api.py` (archivo FastAPI)
- `requirements.txt`

Además, edita o crea un `README.md` y agrega la siguiente metadata:

```yaml
app_port: 8000
```

> Esto le indica a Hugging Face que la aplicación expone el puerto **8000**, usado por la API.

## 5️⃣ Hacer commit y push para desplegar la API

Antes de hacer el *push*, es necesario configurar los **Secrets** de conexión a Databricks para que la API pueda autenticarse correctamente desde el Space.

1. En el panel de tu Space (`nyc-time-api-hf`), ve a:  
   **Settings → Repository secrets**
2. Agrega las siguientes variables (con tus valores reales):

```
DATABRICKS_TOKEN=<tu_token_de_acceso_a_databricks>
DATABRICKS_HOST=<tu_url_host_de_databricks>
```

> 🔒 Estos secretos **no se exponen en el código**, pero estarán disponibles dentro del contenedor como variables de entorno (`os.getenv()`).

---

Una vez configurados los secrets, haz commit y push:

```bash
git add Dockerfile main.py requirements.txt README.md
git commit -m "Deploy API to Hugging Face"
git push origin main
```

Hugging Face iniciará el proceso de **build y deploy** automáticamente.  
Cuando el Space esté en estado **Running**, podrás verificar la API en:

- Documentación:  
  👉 `https://<tu_usuario>-nyc-time-api-hf.hf.space/docs`

- Healthcheck (si está definido):  
  👉 `https://<tu_usuario>-nyc-time-api-hf.hf.space/health`



## 6️⃣ Crear el Space de la UI

1. Entra de nuevo a 👉 [https://huggingface.co/spaces](https://huggingface.co/spaces)
2. Crea otro Space:
   - Nombre: `nyc-time-ui-hf`
   - Runtime: **Docker**
   - Visibilidad: **Public**
3. Clónalo en local:
   ```bash
   git clone https://huggingface.co/spaces/<tu_usuario>/nyc-time-ui-hf
   cd nyc-time-ui-hf
   ```
4. Copia dentro de este repositorio:
   - `Dockerfile`
   - `main.py` (archivo Streamlit)
   - `requirements.txt`
5. En el `README.md`, define el puerto de la UI:
   ```yaml
   app_port: 8501
   ```

> También puedes usar el puerto `7860` (por defecto en HF Spaces para UIs).  
> Si lo haces, ajusta el `CMD` de tu Dockerfile y el `app_port` en el README.

## 7️⃣ Conectar la UI con la API desplegada

En tu archivo `main.py` de la UI, reemplaza cualquier llamada a `localhost` por la **URL pública del Space de la API**.

Por ejemplo:

```python
response = requests.post(
    url="https://<tu_usuario>-nyc-time-api-hf.hf.space/api/v1/predict",
    data=json.dumps(input_dict)
)
```

## 8️⃣ Hacer commit y push de la UI

```bash
git add Dockerfile main.py requirements.txt README.md
git commit -m "Deploy UI to Hugging Face"
git push origin main
```

Cuando la UI esté en estado **Running**, abre:

```
https://<tu_usuario>-nyc-time-ui-hf.hf.space
```

👉 La interfaz ya estará comunicándose con tu API desplegada previamente.

## Quiz la siguiente semana. 18 de Noviembre de 2025 en el horario de clase. Al iniciar? Al final? Lo descubriremos. Será el último quiz del curso.

## Tema: Docker y despliegue con HuggingFace